In [17]:
import tensorflow as tf
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
import numpy as np

Using TensorFlow backend.


In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.model_selection import KFold
import numpy as np
from sklearn import preprocessing 
label_encoder = preprocessing.LabelEncoder()
from sklearn.ensemble import ExtraTreesClassifier
import statistics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.tree import export_graphviz
from sklearn import tree
from IPython.display import SVG
from sklearn.metrics import plot_confusion_matrix
from sklearn.preprocessing import OneHotEncoder

In [51]:
dataset=pd.read_csv("PA_DataSet - Sheet1.csv")

In [52]:
dataset.head()

,Health Insurance Plan,Age,Height,Gender,Weight,Speciality,Treatment,Prescribed Drug,Generic Drug,Addictive,...,Doc_exp,Prev_Med_imp,Side Effects,Insurance Amount,Strength(mg),Dosage(per day),Length of Treatment(days),Therapy Initiated,Reauthorization,Label
0,Platinum,37.0,5.2,Male,45,Cardiac surgeon,Aortic Surgery,Apixaban,Eliquis,no,...,9,YES,yes,95431,875.0,3,14,YES,NO,Accepted
1,Basic,50.0,5.0,Male,43,Electrophysiologist,Minimally Invasive Heart Surgery,Indapamide,Lozol,yes,...,3,YES,yes,54045,200.0,2,5,NO,NO,Rejected
2,Gold,45.0,6.0,Male,55,Breast surgeon,Axillary lymph node dissection (ALND),Docetaxel,NaN,No,...,11,NaN,no,34567,90.0,2,20,NO,YES,Partially Accepted
3,Basic,54.0,5.7,Male,41,Vascular surgeon,Sentinel lymph node biopsy (SLNB),DaunoXome,Daunorubicin,No,...,16,YES,no,79927,200.0,1,8,YES,NO,Accepted
4,Platinum,35.0,6.3,Male,50,Radiologist,Partial nephrectomy,Zykadia,Ceritinib,No,...,8,NO,yes,95341,25.0,1,10,NO,NO,Accepted


In [53]:
dataset.describe()

,Age,Height,Weight,Pres_Med_cost,Doc_exp,Insurance Amount,Strength(mg),Dosage(per day),Length of Treatment(days)
count,998.000000,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000
mean,44.871743,5.565165,61.088088,60389.533534,8.879880,74158.722723,310.518018,2.234234,17.585586
std,20.356097,1.212711,18.291567,29778.824937,4.963149,14305.804351,308.584925,0.954469,10.376211
min,0.000000,1.000000,24.000000,1534.000000,0.000000,34567.000000,6.250000,1.000000,4.000000
25%,32.000000,5.400000,45.000000,57927.000000,4.000000,61531.500000,40.000000,2.000000,10.000000
50%,47.000000,5.900000,63.000000,68581.000000,9.000000,73728.000000,160.000000,2.000000,14.000000
75%,60.000000,6.200000,77.000000,81591.500000,13.000000,86249.000000,560.000000,3.000000,21.000000
max,89.000000,6.900000,92.000000,94997.000000,17.000000,99621.000000,1000.000000,4.000000,42.000000


In [54]:
dataset.isnull().sum()

Health Insurance Plan          0
Age                            1
Height                         0
Gender                         0
Weight                         0
Speciality                     0
Treatment                      0
Prescribed Drug                0
Generic Drug                  47
Addictive                      2
Pres_Med_cost                  0
Alter_Med_cost                 0
Doc_exp                        0
Prev_Med_imp                 409
Side Effects                 170
Insurance Amount               0
Strength(mg)                   0
Dosage(per day)                0
Length of Treatment(days)      0
Therapy Initiated              0
Reauthorization                0
Label                          0
dtype: int64

In [55]:
dataset["Generic Drug"]=dataset["Generic Drug"].fillna("Not Generic")
dataset["Addictive"]=dataset["Addictive"].fillna("NO")
dataset["Doc_exp"]=dataset["Doc_exp"].fillna(dataset["Doc_exp"].median())
dataset["Age"]=dataset["Age"].fillna(dataset["Age"].median())
dataset["Prev_Med_imp"]=dataset["Prev_Med_imp"].fillna("Not Available")
dataset["Side Effects"]=dataset["Side Effects"].fillna("No")
dataset["Addictive"]=dataset["Addictive"].str.upper()
dataset["Prev_Med_imp"]=dataset["Prev_Med_imp"].str.upper()
dataset["Side Effects"]=dataset["Side Effects"].str.upper()
dataset["Addictive"]=dataset["Addictive"].str.upper()

In [56]:
dataset["Label"].replace({"A": "Accepted", "Rejected (P)": "Rejected","Partially Accepted":"Accepted"}, inplace=True)
dataset["Label"].unique()

array(['Accepted', 'Rejected'], dtype=object)

In [57]:
col=['Label', 'Gender', 'Addictive', 'Prev_Med_imp', 'Side Effects','Reauthorization','Therapy Initiated']
for c in col:
    dataset[c]=label_encoder.fit_transform(dataset[c])

In [58]:
dataset=dataset.drop(["Speciality","Prescribed Drug","Generic Drug","Treatment"],axis=1)

In [59]:
dataset.head()

,Health Insurance Plan,Age,Height,Gender,Weight,Addictive,Pres_Med_cost,Alter_Med_cost,Doc_exp,Prev_Med_imp,Side Effects,Insurance Amount,Strength(mg),Dosage(per day),Length of Treatment(days),Therapy Initiated,Reauthorization,Label
0,Platinum,37.0,5.2,2,45,0,93689,95000,9,2,1,95431,875.0,3,14,1,0,0
1,Basic,50.0,5.0,2,43,1,67959,92258,3,2,1,54045,200.0,2,5,0,0,1
2,Gold,45.0,6.0,2,55,0,59544,41517,11,1,0,34567,90.0,2,20,0,1,0
3,Basic,54.0,5.7,2,41,0,61165,78394,16,2,0,79927,200.0,1,8,1,0,0
4,Platinum,35.0,6.3,2,50,0,91680,89750,8,0,1,95341,25.0,1,10,0,0,0


In [60]:
hp_dummies=pd.get_dummies(dataset['Health Insurance Plan'], prefix='HPlan')
dataset = pd.concat([hp_dummies, dataset], axis=1).drop(['Health Insurance Plan'], axis=1)
dataset.head()

,HPlan_Basic,HPlan_Gold,HPlan_Platinum,Age,Height,Gender,Weight,Addictive,Pres_Med_cost,Alter_Med_cost,Doc_exp,Prev_Med_imp,Side Effects,Insurance Amount,Strength(mg),Dosage(per day),Length of Treatment(days),Therapy Initiated,Reauthorization,Label
0,0,0,1,37.0,5.2,2,45,0,93689,95000,9,2,1,95431,875.0,3,14,1,0,0
1,1,0,0,50.0,5.0,2,43,1,67959,92258,3,2,1,54045,200.0,2,5,0,0,1
2,0,1,0,45.0,6.0,2,55,0,59544,41517,11,1,0,34567,90.0,2,20,0,1,0
3,1,0,0,54.0,5.7,2,41,0,61165,78394,16,2,0,79927,200.0,1,8,1,0,0
4,0,0,1,35.0,6.3,2,50,0,91680,89750,8,0,1,95341,25.0,1,10,0,0,0


In [61]:
le_name_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print(le_name_mapping)
#  Accepted:0, Rejected:1

{'NO': 0, 'YES': 1}


In [62]:
X=dataset.iloc[:,0:19]
Y=dataset.iloc[:,19:20]
X.isnull().sum()


HPlan_Basic                  0
HPlan_Gold                   0
HPlan_Platinum               0
Age                          0
Height                       0
Gender                       0
Weight                       0
Addictive                    0
Pres_Med_cost                0
Alter_Med_cost               0
Doc_exp                      0
Prev_Med_imp                 0
Side Effects                 0
Insurance Amount             0
Strength(mg)                 0
Dosage(per day)              0
Length of Treatment(days)    0
Therapy Initiated            0
Reauthorization              0
dtype: int64

In [63]:
Y.isnull().sum()

Label    0
dtype: int64

In [64]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [65]:
X_train.head()

,HPlan_Basic,HPlan_Gold,HPlan_Platinum,Age,Height,Gender,Weight,Addictive,Pres_Med_cost,Alter_Med_cost,Doc_exp,Prev_Med_imp,Side Effects,Insurance Amount,Strength(mg),Dosage(per day),Length of Treatment(days),Therapy Initiated,Reauthorization
615,0,1,0,0.0,1.0,1,29,0,75381,47663,5,0,1,82598,300.0,1,30,1,0
949,1,0,0,48.0,6.3,2,70,0,6183,7376,12,1,1,68674,40.0,4,42,0,0
864,1,0,0,73.0,6.3,2,81,0,5863,4826,14,1,0,84009,1000.0,2,21,0,1
821,0,1,0,36.0,5.7,2,77,0,3692,4302,14,2,0,98977,125.0,1,5,0,0
634,0,1,0,0.0,1.0,1,88,0,55470,50583,12,2,1,73516,875.0,2,20,1,1


In [66]:
Y_train.head()

,Label
615,1
949,0
864,1
821,0
634,0


In [67]:
model = Sequential()
model.add(Dense(12, input_dim=19, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [68]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [81]:
model.fit(X_train, Y_train, epochs=190, batch_size=10)

Epoch 1/190
799/799 [==============================] - 0s 174us/step - loss: 0.5883 - accuracy: 0.6195
Epoch 2/190
799/799 [==============================] - 0s 153us/step - loss: 0.5872 - accuracy: 0.6395
Epoch 3/190
799/799 [==============================] - 0s 153us/step - loss: 0.5832 - accuracy: 0.6245
Epoch 4/190
799/799 [==============================] - 0s 148us/step - loss: 0.6249 - accuracy: 0.6295
Epoch 5/190
799/799 [==============================] - 0s 144us/step - loss: 0.5767 - accuracy: 0.6283
Epoch 6/190
799/799 [==============================] - 0s 153us/step - loss: 0.5830 - accuracy: 0.6408
Epoch 7/190
799/799 [==============================] - 0s 150us/step - loss: 0.5714 - accuracy: 0.6395
Epoch 8/190
799/799 [==============================] - 0s 154us/step - loss: 0.5949 - accuracy: 0.6370
Epoch 9/190
799/799 [==============================] - 0s 156us/step - loss: 0.6120 - accuracy: 0.6220
Epoch 10/190
799/799 [==============================] - 0s 150us/step - l

In [82]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 12)                240       
_________________________________________________________________
dense_11 (Dense)             (None, 8)                 104       
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 9         
Total params: 353
Trainable params: 353
Non-trainable params: 0
_________________________________________________________________


In [87]:
_, accuracy = model.evaluate(X_train, Y_train)
print('Accuracy: %.2f' % (accuracy*100))

799/799 [==============================] - 0s 33us/step
Accuracy: 80.23
